<h2> Movimiento Motor </h2>
<h4>1. Establecer la conexión y obtener los controladores (handlers)</h4>

In [28]:
import sim
import numpy as np
import math
import time

def connect(port):
# Establece la conexión a COPPELIA
# El port debe coincidir con el puerto de conexión en VREP  -- DALE AL PLAY !!!
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID

# Conectarse al servidor de COPPELIA
# *** _Hay que ejecutarlo cada vez que se reinicia la simulación ***
clientID = connect(19999)
ret,nema23=sim.simxGetObjectHandle(clientID,'nema23',sim.simx_opmode_blocking)

conectado a 19999


<h4> 3. Posición de las articulaciones?</h4>

In [29]:
# leemos la posición de Joint1, en radianes.
returnCode, radNema = sim.simxGetJointPosition(clientID, nema23, sim.simx_opmode_blocking)
degNema=radNema *180/np.pi
print("Radiants:", radNema, "Graus:", degNema)


Radiants: 0.017778964713215828 Graus: 1.018659642179285


<h4> 4. Conntrol de velocidad del robot</h4>

In [30]:
MAX_DEG = 60
MIN_DEG = 0
MAX_VEL = 1
STOP = 0

VOL_MIN=50
VOL_MAX=90

RPM_MIN=5
RPM_MAX=30

RADI_MIN=1
RADI_MAX=3

ERROR_MESSAGE= "Valor incorrecte. Siusplau, introdueixi un valor entre"


# Per assigar els parametres del robot, sha utilitzat criteris aproximats, logics i proporcionals als angles maxims i velocitat 
# maxims del robot. PEr fer'ho realista i segur, hauriem de coneixer les caracteristiques mediques dún ventilador manual extern, 
# quines son les necessitats pulmonars del cos humà

def movimentParametritzat(volAire, rpm, radi):
    angle=0 
    
    # VOLUM AIRE = 50-90
    while volAire < VOL_MIN or volAire > VOL_MAX:
        print(ERROR_MESSAGE, VOL_MIN ," i ", VOL_MAX)
        volAire = int(input())               
        
    maxDeg = volAire-30
    
    # RPM = 5-30
    while rpm < RPM_MIN or rpm > RPM_MAX:
        print(ERROR_MESSAGE, RPM_MIN ," i ", RPM_MAX)
        rpm = int(input())
        
    vel = ((rpm-RPM_MIN)*(100/(RPM_MAX-RPM_MIN)))/100
        
         
    # RADI = 1,2,3
    while radi < RADI_MIN or radi > RADI_MAX:
        print(ERROR_MESSAGE, RADI_MIN ," i ", RADI_MAX)
        radi = int(input())
        
    velInsp = vel
    velExp = vel
    
    if radi != RADI_MIN:
        if radi == RADI_MAX:
            velInsp = vel*0.5
        else:
            velExp = vel*0.5  
            
    ## - Funcionalitat - ##
    for i in range(10):
        while(angle<maxDeg):
            sim.simxSetJointTargetVelocity(clientID, nema23, velInsp, sim.simx_opmode_oneshot)
            retCode,radNema = sim.simxGetJointPosition(clientID, nema23, sim.simx_opmode_blocking)
            angle=radNema *180/np.pi

        sim.simxSetJointTargetVelocity(clientID, nema23, STOP, sim.simx_opmode_oneshot)

        while(angle>MIN_DEG):
            sim.simxSetJointTargetVelocity(clientID, nema23, -velExp, sim.simx_opmode_oneshot)
            retCode,radNema = sim.simxGetJointPosition(clientID, nema23, sim.simx_opmode_blocking)
            angle=radNema *180/np.pi

        sim.simxSetJointTargetVelocity(clientID, nema23, STOP, sim.simx_opmode_oneshot)

In [4]:

movimentParametritzat(50, 25, 2) #Test 1

movimentParametritzat(50, 15, 2) #Test 2

movimentParametritzat(50, 30, 2) #Test 3


Valor incorrecte. Siusplau, introdueixi un valor entre 50  i  90
20
Valor incorrecte. Siusplau, introdueixi un valor entre 50  i  90
50
Valor incorrecte. Siusplau, introdueixi un valor entre 5  i  30
20
Valor incorrecte. Siusplau, introdueixi un valor entre 1  i  3
1


In [31]:
movimentParametritzat(70, 22, 1) #Test 1
movimentParametritzat(70, 30, 1) #Test 1
movimentParametritzat(70, 15, 1) #Test 2
